# WroldMap with marked data

## Refs

- [spatial-visualizations](https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf)
- [folium docs](https://python-visualization.github.io/folium/)
  - [examples](https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/)
  - [notebooks](https://nbviewer.jupyter.org/github/python-visualization/folium_contrib/tree/master/notebooks/)

## Setup Envs

In [20]:
%conda install folium -c conda-forge

Solving environment: ...working... 
  - https://repo.continuum.io/pkgs/main/win-64/asn1crypto-0.24.0-py36_0.tar.bz2::asn1crypto-0.24.0-py36_0, https://repo.continuum.io/pkgs/main/win-64/libiconv-1.15-h1df5818_7.tar.bz2::libiconv-1.15-h1df5818_7, https://repo.continuum.io/pkgs/main/win-64/lzo-2.10-h6df0209_2.tar.bz2::lzo-2.10-h6df0209_2, https://repo.continuum.io/pkgs/main/win-64/yaml-0.1.7-hc54c509_2.tar.bz2::yaml-0.1.7-hc54c509_2
  - defaults::lzo-2.10-h6df0209_2, https://repo.continuum.io/pkgs/main/win-64/asn1crypto-0.24.0-py36_0.tar.bz2::asn1crypto-0.24.0-py36_0, https://repo.continuum.io/pkgs/main/win-64/libiconv-1.15-h1df5818_7.tar.bz2::libiconv-1.15-h1df5818_7, https://repo.continuum.io/pkgs/main/win-64/yaml-0.1.7-hc54c509_2.tar.bz2::yaml-0.1.7-hc54c509_2
  - defaults::libiconv-1.15-h1df5818_7, https://repo.continuum.io/pkgs/main/win-64/asn1crypto-0.24.0-py36_0.tar.bz2::asn1crypto-0.24.0-py36_0, https://repo.continuum.io/pkgs/main/win-64/lzo-2.10-h6df0209_2.tar.bz2::lzo-2.10-h6df

In [35]:
import pandas as pd
import folium

In [36]:
envs = {
    'locations': 'Volcanoes.txt',
    'mapfile': 'volcanoes at map.html',
    'geopopfile': 'world.json'
}
envs

{'locations': 'Volcanoes.txt',
 'mapfile': 'volcanoes at map.html',
 'geopopfile': 'world.json'}

## Read data with Pandas

In [37]:
locdata = pd.read_csv(envs['locations'])
latitues = list(locdata["LAT"])
longitudes = list(locdata["LON"])
elevations =list(locdata["ELEV"])
colors = map(lambda x: 'green' if x<1000 else ('orange' if x<3000 else 'red' ), elevations)

In [38]:
locdata

,VOLCANX020,NUMBER,NAME,LOCATION,STATUS,ELEV,TYPE,TIMEFRAME,LAT,LON
0,509.0,1201-01=,Baker,US-Washington,Historical,3285.0,Stratovolcanoes,D3,48.776798,-121.810997
1,511.0,1201-02-,Glacier Peak,US-Washington,Tephrochronology,3213.0,Stratovolcano,D4,48.111801,-121.111000
2,513.0,1201-03-,Rainier,US-Washington,Dendrochronology,4392.0,Stratovolcano,D3,46.869801,-121.751000
3,515.0,1201-05-,St. Helens,US-Washington,Historical,2549.0,Stratovolcano,D1,46.199799,-122.181000
4,516.0,1201-04-,Adams,US-Washington,Tephrochronology,3742.0,Stratovolcano,D6,46.205799,-121.490997
5,517.0,1201-06-,West Crater,US-Washington,Radiocarbon,1329.0,Volcanic field,D7,45.879799,-122.081001
6,518.0,1201-07-,Indian Heaven,US-Washington,Radiocarbon,1806.0,Shield volcanoes,D7,45.929798,-121.820999
7,519.0,1202-01-,Hood,US-Oregon,Historical,3426.0,Stratovolcano,D3,45.373798,-121.691002
8,521.0,1202-02-,Jefferson,US-Oregon,Varve Count,3199.0,Stratovolcano,D6,44.691799,-121.801002
9,522.0,1202-03-,Blue Lake Crater,US-Oregon,Radiocarbon,1230.0,Maar,D7,44.419800,-121.771004


## Spacial Visualize with Folium

### load map

In [39]:
worldmap = folium.Map(location=[40.693943, -73.985880], 
                      zoom_start=6, control_scale=True, tiles='Mapbox Bright')

In [40]:
worldmap

### create Layers

In [41]:
fg_volcanoes = folium.FeatureGroup(name="Volcanoes")
for lt, ln, el, color in zip(latitues, longitudes, elevations, colors):
    fg_volcanoes.add_child(folium.CircleMarker(location=[lt, ln], radius = 6, popup=str(el)+"m",
    fill_color=color, fill=True,  color = 'grey', fill_opacity=0.7))

In [42]:
fg_population = folium.FeatureGroup(name="Population")

data_geopop = open(envs['geopopfile'], 'r', encoding='utf-8-sig').read()

fg_population.add_child(folium.GeoJson(data=data_geopop,
style_function=lambda x: {'fillColor':'green' if x['properties']['POP2005'] < 10000000
else 'orange' if 10000000 <= x['properties']['POP2005'] < 20000000 else 'red'}))

In [43]:
worldmap.add_child(fg_volcanoes)
worldmap.add_child(fg_population)
worldmap.add_child(folium.LayerControl())

worldmap.save(envs['mapfile'])

### Folim Playgroud

In [61]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=15):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [62]:
home = [30.203292, 120.188228]
homemap = generateBaseMap(home)

In [63]:
homemap